<a href="https://colab.research.google.com/github/Srinivas-8612/Data_Structures/blob/main/B_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------
# B-TREE MENU DRIVEN PROGRAM
# ----------------------------

class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t               # Minimum degree
        self.leaf = leaf         # True if node is leaf
        self.keys = []           # List of keys
        self.children = []       # List of children

    def traverse(self):
        """Return inorder traversal of the subtree."""
        result = []
        for i in range(len(self.keys)):
            if not self.leaf:
                result.extend(self.children[i].traverse())
            result.append(self.keys[i])
        if not self.leaf:
            result.extend(self.children[len(self.keys)].traverse())
        return result

    def search(self, k):
        """Search key k in the subtree."""
        i = 0
        while i < len(self.keys) and k > self.keys[i]:
            i += 1

        if i < len(self.keys) and self.keys[i] == k:
            return True

        if self.leaf:
            return False
        return self.children[i].search(k)

    def insert_non_full(self, k):
        """Insert key k into a non-full node."""
        i = len(self.keys) - 1

        if self.leaf:
            self.keys.append(0)
            while i >= 0 and k < self.keys[i]:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
        else:
            while i >= 0 and k < self.keys[i]:
                i -= 1
            i += 1
            if len(self.children[i].keys) == 2 * self.t - 1:
                self.split_child(i)
                if k > self.keys[i]:
                    i += 1
            self.children[i].insert_non_full(k)

    def split_child(self, i):
        """Split a full child node."""
        t = self.t
        y = self.children[i]
        z = BTreeNode(y.t, y.leaf)

        z.keys = y.keys[t:]
        mid_key = y.keys[t - 1]
        y.keys = y.keys[:t - 1]

        if not y.leaf:
            z.children = y.children[t:]
            y.children = y.children[:t]

        self.children.insert(i + 1, z)
        self.keys.insert(i, mid_key)

    def remove(self, k):
        """Remove a key from the subtree rooted with this node."""
        idx = self.find_key(k)
        if idx < len(self.keys) and self.keys[idx] == k:
            if self.leaf:
                self.keys.pop(idx)
            else:
                self.remove_internal_node(k, idx)
        else:
            if self.leaf:
                print(f"Key {k} not found in the B-Tree.")
                return
            flag = idx == len(self.keys)
            if len(self.children[idx].keys) < self.t:
                self.fill(idx)
            if flag and idx > len(self.keys):
                self.children[idx - 1].remove(k)
            else:
                self.children[idx].remove(k)

    def find_key(self, k):
        """Find the first index greater than or equal to k."""
        idx = 0
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        return idx

    def remove_internal_node(self, k, idx):
        if len(self.children[idx].keys) >= self.t:
            pred = self.get_pred(idx)
            self.keys[idx] = pred
            self.children[idx].remove(pred)
        elif len(self.children[idx + 1].keys) >= self.t:
            succ = self.get_succ(idx)
            self.keys[idx] = succ
            self.children[idx + 1].remove(succ)
        else:
            self.merge(idx)
            self.children[idx].remove(k)

    def get_pred(self, idx):
        cur = self.children[idx]
        while not cur.leaf:
            cur = cur.children[-1]
        return cur.keys[-1]

    def get_succ(self, idx):
        cur = self.children[idx + 1]
        while not cur.leaf:
            cur = cur.children[0]
        return cur.keys[0]

    def fill(self, idx):
        if idx != 0 and len(self.children[idx - 1].keys) >= self.t:
            self.borrow_from_prev(idx)
        elif idx != len(self.keys) and len(self.children[idx + 1].keys) >= self.t:
            self.borrow_from_next(idx)
        else:
            if idx != len(self.keys):
                self.merge(idx)
            else:
                self.merge(idx - 1)

    def borrow_from_prev(self, idx):
        child = self.children[idx]
        sibling = self.children[idx - 1]
        child.keys.insert(0, self.keys[idx - 1])
        if not child.leaf:
            child.children.insert(0, sibling.children.pop())
        self.keys[idx - 1] = sibling.keys.pop()

    def borrow_from_next(self, idx):
        child = self.children[idx]
        sibling = self.children[idx + 1]
        child.keys.append(self.keys[idx])
        if not child.leaf:
            child.children.append(sibling.children.pop(0))
        self.keys[idx] = sibling.keys.pop(0)

    def merge(self, idx):
        child = self.children[idx]
        sibling = self.children[idx + 1]
        child.keys.append(self.keys[idx])
        child.keys.extend(sibling.keys)
        if not child.leaf:
            child.children.extend(sibling.children)
        self.keys.pop(idx)
        self.children.pop(idx + 1)


class BTree:
    def __init__(self, t):
        self.t = t
        self.root = None

    def insert(self, k):
        """Insert a key into the B-Tree."""
        if self.root is None:
            self.root = BTreeNode(self.t, True)
            self.root.keys.append(k)
        else:
            if len(self.root.keys) == 2 * self.t - 1:
                s = BTreeNode(self.t, False)
                s.children.insert(0, self.root)
                s.split_child(0)
                i = 0
                if s.keys[0] < k:
                    i += 1
                s.children[i].insert_non_full(k)
                self.root = s
            else:
                self.root.insert_non_full(k)

    def search(self, k):
        """Search for a key."""
        if self.root is None:
            return False
        return self.root.search(k)

    def delete(self, k):
        """Delete a key from the tree."""
        if not self.root:
            print("The tree is empty!")
            return
        self.root.remove(k)
        if len(self.root.keys) == 0:
            if self.root.leaf:
                self.root = None
            else:
                self.root = self.root.children[0]

    def traverse(self):
        """Traverse the tree in sorted order."""
        if self.root is not None:
            return self.root.traverse()
        return []


# ---------------- MENU DRIVEN PROGRAM ----------------

if __name__ == "__main__":
    t = int(input("Enter the minimum degree (t) of the B-Tree: "))
    btree = BTree(t)

    while True:
        print("\n========== B-TREE OPERATIONS ==========")
        print("1. Insert Key")
        print("2. Delete Key")
        print("3. Search Key")
        print("4. Display (Inorder Traversal)")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            key = int(input("Enter key to insert: "))
            btree.insert(key)
            print(f"✅ Key {key} inserted successfully!")

        elif choice == '2':
            key = int(input("Enter key to delete: "))
            btree.delete(key)
            print(f"🗑️ Key {key} deleted (if existed).")

        elif choice == '3':
            key = int(input("Enter key to search: "))
            found = btree.search(key)
            if found:
                print(f"🔍 Key {key} found in the B-Tree.")
            else:
                print(f"❌ Key {key} not found in the B-Tree.")

        elif choice == '4':
            print("🌳 B-Tree Traversal (sorted order):", btree.traverse())

        elif choice == '5':
            print("👋 Exiting program. Goodbye!")
            break

        else:
            print("⚠️ Invalid choice! Please enter a number between 1–5.")


Enter the minimum degree (t) of the B-Tree: 4

========== B-TREE OPERATIONS ==========
1. Insert Key
2. Delete Key
3. Search Key
4. Display (Inorder Traversal)
5. Exit
Enter your choice (1-5): 1
Enter key to insert: 23
✅ Key 23 inserted successfully!

========== B-TREE OPERATIONS ==========
1. Insert Key
2. Delete Key
3. Search Key
4. Display (Inorder Traversal)
5. Exit
Enter your choice (1-5): 3
Enter key to search: 23
🔍 Key 23 found in the B-Tree.

========== B-TREE OPERATIONS ==========
1. Insert Key
2. Delete Key
3. Search Key
4. Display (Inorder Traversal)
5. Exit
Enter your choice (1-5): 4
🌳 B-Tree Traversal (sorted order): [23]

========== B-TREE OPERATIONS ==========
1. Insert Key
2. Delete Key
3. Search Key
4. Display (Inorder Traversal)
5. Exit
